# Problem formulation

The original vector:

<!-- $X_n = (ξ^{IMU}_n, ν^{IMU}_n, p_n, β^a_n, β^ω_n, ξ^{car}, p^{car})$ informular we need to put it in skew matrix form, handly differentiate and use analytic result to update in vertor form -->
$$X_n = (R^{IMU}_n, v^{IMU}_n, p_n, β^a_n, β^ω_n, R^{car}, p^{car})$$


#### Where

+ Rotation matrix of IMU w.r.t global in global frame (the $ξ^{IMU}$ $\in \mathbb{R}^3$ represented as Euler rotation):

$R^{IMU} = exp(ξ^{IMU}∧)$

+ Rotation matrix of car's center w.r.t IMU in IMU frame, aka misalignment between car and IMU  (the ξ $^{car}$ $\in \mathbb{R}^3$ represented as Euler rotation):

$R^{car} = exp(ξ^{car}∧)$

$\text{real measurement IMU} =  ω^{IMU} \rightarrow \text{ideal value}= ω_n = ω^{IMU}_n - β^w_n$

$real measurement IMU = a^{IMU} \rightarrow \text{ideal value}= a_n = a^{IMU}_n - β^a_n$


As they explain in the paper(APPENDIX A.1), the variables $\chi_{IMU} = (R^{IMU}_n, v^{IMU}_n, p_n)$ are embedded in the Lie group SE2(3), and they can map the increment term (or uncertainties) $ξ_n \in \mathbb{R}^5 \longmapsto ξ^∧_n \in se2(3)$. Then they find the Jacobian matrix of obervation H with respect to that linearized error $e_n \in \mathbb{R}^{21}$ (equation (21) in paper): 

$$e_n = (ξ^{IMU}_n, e^{b}_n, ξ^{R^c}_n, ξ^{p^c}_n)$$ 

Here, this increment $e_n$ was mixed of $SE_2(3)$ (for IMU), $\mathbb{R}^6$ (for bias IMU), $SO(3)$ (axis misalignment between IMU-car), $\mathbb{R}^3$ (translation vector between IMU-car). 

## Question 1: Then how they can find Jacobian matrix in their paper? 
    
we supposed:

+ for  $\mathbb{R}^6$ and $\mathbb{R}^3$ term they find Jacobian as normal, (differentiate w.r.t vector); 

+ for Lie group $SE_2(3)$ and  $SO(3)$ term, they find it using some magic of Lie group like: ${\frac {dR}{dξ}}$  or ${\frac {d(R_1\times R_2)}{dξ}}$, what is the formula to do this? should we do it based on the formula in Question 3?

Then they can combine them to have Jacobian matrix H in formula (37)

## Question 2: How we can find new Jacobian matrix if we add one more pseudo-measurement? 

The new additional pseudo-measurement: The acceleration of car on lateral axis should be equal to centrifugal acceleration


$h(X) = a_y^{car} - ν^{for} · ω_z^{car}$

### For $a_y^{car}$
$a^{car}$ is acceleration of car's body w.r.t global O but align along car frame axis

| $$a_{B/O}=$$  | $$a_{A/O}$$ | $$\dot{\omega} \times r_{AB}$$ | $$\omega \times \omega \times r_{AB}$$ |
|--------|-----------|-----------|-----------|
| $$a^{car}=$$ | $$R_n^{car T}(a_n)$$ | $$R_n^{car T}( \dot{\omega_{n}} \times p_n^c)$$ | $$R_n^{car T} (\omega_{n}\times(\omega_{n}\times p_n^c))$$ |
| Explanation | Acceleration transformation from IMU to car frame. | Tangent acceleration of car around IMU (as we understand, $p_n^c$ was defined in car's frame, hence this term will be in car's frame)| Coriolis term (as we understand, $p_n^c$ was defined in car's frame, hence this term will be in car's frame)|



<!--$a^{car} = R_n^{car T} R_n^{IMU \space T} (a_n) + \dot{\omega_{n}} \times p_n^c + \omega_{n}\times(\omega_{n}\times p_n^c)$ -->

+ For now, lets assume  $\dot{\omega_{IMU}} \times p_n^c = 0$, the car doesnt rotate too fast (ofcourse when the car does drift), otherwise we need to add vector of angular acceleration as estimated state

$\Leftrightarrow a^{car} = R_n^{car T} ( (a_n) + \omega_{n}\times(\omega_{n}\times p_n^c))$


$a_y^{car} = (0,1,0)a^{car}$

#### For $ω_z^{car}$

+ Because car and IMU was fixed, they are rotating together.

$ω^{car \text{\&} imu} = ω_n = ω_n^{IMU} - β_n^ω$

+ But we have misalignment of axis between car and IMU, we need to transform and find the yaw angular velocity in car frame $(\omega_z)$

$ω^{car} = (0,0,1)R_n^{car\space T} (ω_n^{IMU} - β_n^ω)$

#### For $v^{for}$

$v^{for} = (1,0,0)v^{car}$

$v^{car} = R_n^{car\space T} ( R_n^{IMU \space T} v_n^{IMU} + ω_n^c × p_n^c)$



 
## Question 3: how to differentiate $\frac{d(R)}{d \alpha} = \frac{d(exp(θ^∧))}{d \alpha}$?

### our current solution (if $\alpha$ is scalar), but we are not sure this is correct or not!!!

+ we want to use Duhamel's formula

$${\frac {d}{dt}}e^{X(t)}=e^{X(t)}{\frac {1-e^{-\mathrm {ad} _{X}}}{\mathrm {ad} _{X}}}{\frac {dX(t)}{dt}}$$

Where 

+ X is skew matrix:

$$X = (\omega)_{\times} = \begin{pmatrix}
0 & -\omega_z & \omega_y \\
\omega_z & 0 & -\omega_x \\
-\omega_y & -\omega_x & 0
\end{pmatrix}$$

+ $t$ now is $\omega_z$

+ The term ${ad} _{X}$ is Adjoint in SO(3):

$$\Rightarrow  {ad} _{X} = X $$


Then we have the result:


$${\frac {d}{\omega_z}}R={\frac {d}{\omega_z}}e^{(\omega)_{\times}}=e^{(\omega)_{\times}}{\frac {1-e^{-(\omega)_{\times}}}{(\omega)_{\times}}}{\frac {d(\omega)_{\times}}{\omega_z}}$$
$${\Leftrightarrow \frac {d}{\omega_z}}e^{(\omega)_{\times}}=e^{(\omega)_{\times}}{\frac {1-e^{-(\omega)_{\times}}}{(\omega)_{\times}}}
\times
\begin{bmatrix}
0 & -1 & 0 \\
1 & 0 & 0 \\
0 & 0 & 0
\end{bmatrix}$$


### is there any formula to do this if $\alpha$ is vector $\in \mathbb{R}^3$


In [1]:
import sympy as smp 


R_c_n = smp

# formula 14
v_c_n = smp.zeros(3,1)
v_c_n = R_c_n.T @ R_imu_n.T @ v_imu_n + skew_matrix(omega_n) @ p_c_n


AttributeError: module 'sympy' has no attribute 'T'

In [ ]:
import sympy as sp

def skew_matrix(_vector):
    return sp.Matrix([[0, -_vector[2], _vector[1]],[_vector[2], 0, -_vector[0]],[-_vector[1], _vector[0],0]])

# Define symbolic variables
# in frame of IMU
omega_imu, omega_ideal, bias_w, ww = sp.symbols(r'\omega_{imu} \omega_{ideal} b_{\omega} w_{\omega}')
a_imu, a_ideal, bias_a, wa = sp.symbols(r'a_{imu} a_{ideal} b_a w_a')

# for random walk model IMU nosie
ww_b = sp.symbols(r'w^b_{n,\omega}')
ww_a = sp.symbols(r'w_b_{n,a}')

# for kinematics propagation
R_imu_n, R_imu_n1 = sp.symbols(r'R_{imu,n} R_{imu,n+1}')
v_imu_n, v_imu_n1, g, dt = sp.symbols(r'v_{imu,n} v_{imu,n+1} g dt')
p_imu_n, p_imu_n1 = sp.symbols(r'p_{imu,n} p_{imu,n+1}')

# for measurement
y = sp.symbols('y')

# estimated velocity
vfor, vlat, vup = sp.symbols(r'v_{for} v_{lat} v_{up}')
vc_n = sp.Matrix([vfor, vlat, vup])
Rcn = sp.symbols('R^c_n')


# (1) IMU angular velocity measurement model
eq1 = sp.Eq(omega_imu, omega_ideal + bias_w + ww)
print("(1) IMU angular velocity measurement model:")
sp.pprint(eq1)

# (2) IMU acceleration measurement model
eq2 = sp.Eq(a_imu, a_ideal + bias_a + wa)
print("\n(2) IMU acceleration measurement model:")
sp.pprint(eq2)

# (3) Bias random walk model for angular velocity
eq3 = sp.Eq(bias_w  , bias_w + ww_b)
print("\n(3) Bias random walk model for angular velocity:")
sp.pprint(eq3)

# (4) Bias random walk model for acceleration
eq4 = sp.Eq(bias_a, bias_a + ww_a)
print("\n(4) Bias random walk model for acceleration:")
sp.pprint(eq4)

# (5) IMU orientation update
# Note: In SymPy, you would typically represent rotation matrices
# using a symbolic representation of the rotation itself (e.g., Euler angles
# or quaternions) and then create the rotation matrix from that.
# Here, I'm leaving it in symbolic form to match the paper, but in
# a real implementation, you'd use a specific representation.
# Also, the cross-product as an exponent is symbolic. In practice,
# you'd use Rodrigues' rotation formula to convert the angular velocity
# and time step into a rotation matrix.
eq5 = sp.Eq(R_imu_n, R_imu_n * sp.exp(omega_ideal*dt)) #Simplified skew-symetric matrix to display properly
print("\n(5) IMU orientation update:")
sp.pprint(eq5)

# (6) IMU velocity update
eq6 = sp.Eq(v_imu_n1, v_imu_n + (R_imu_n * a + g) * dt)
print("\n(6) IMU velocity update:")
sp.pprint(eq6)

# (7) IMU position update
eq7 = sp.Eq(p_imu_n1, p_imu_n + v_imu_n * dt)
print("\n(7) IMU position update:")
sp.pprint(eq7)

# (12) Rotation update
eq12 = sp.Eq(Rcn, Rcn * sp.exp()) #Simplified skew-symetric matrix to display properly
print("\n(12) Rotation update:")
sp.pprint(eq12)

# (14) Vehicle velocity in car frame
pc_n = sp.symbols('pc_n')
omega_n = sp.symbols('omega_n')
vimu_n = sp.symbols('vimu_n')
Rimu_n = sp.symbols('Rimu_n')
eq14 = sp.Eq(vc_n, sp.transpose(Rcn) * sp.transpose(Rimu_n) * vimu_n + sp.Matrix([0,0,0])) #Simplified skew-symetric matrix
print("\n(14) Vehicle velocity in car frame:")
sp.pprint(eq14)

# (15) Pseudo-measurement
n_lat, n_up = sp.symbols('n_lat n_up')
eq15 = sp.Eq(sp.Matrix([y, y]), sp.Matrix([vlat, vup]) + sp.Matrix([n_lat, n_up]))
print("\n(15) Pseudo-measurement:")
sp.pprint(eq15)


# 

In [ ]:
    def update(self, Rot, v, p, b_omega, b_acc, Rot_c_i, t_c_i, P, u, i, measurement_cov):
        # orientation of body frame
        Rot_body = Rot.dot(Rot_c_i)
        # velocity in imu frame
        v_imu = Rot.T.dot(v)
        # velocity in body frame
        v_body = Rot_c_i.T.dot(v_imu)
        # velocity in body frame in the vehicle axis
        v_body += self.skew(t_c_i).dot(u[:3] - b_omega)
        Omega = self.skew(u[:3] - b_omega)

        # Jacobian w.r.t. car frame
        H_v_imu = Rot_c_i.T.dot(self.skew(v_imu))
        H_t_c_i = -self.skew(t_c_i)

        H = np.zeros((2, self.P_dim))
        H[:, 3:6] = Rot_body.T[1:]
        H[:, 15:18] = H_v_imu[1:]
        H[:, 9:12] = H_t_c_i[1:]
        H[:, 18:21] = -Omega[1:]
        r = - v_body[1:]
        R = np.diag(measurement_cov)
        Rot_up, v_up, p_up, b_omega_up, b_acc_up, Rot_c_i_up, t_c_i_up, P_up = \
            self.state_and_cov_update(Rot, v, p, b_omega, b_acc, Rot_c_i, t_c_i, P, H, r, R)
        return Rot_up, v_up, p_up, b_omega_up, b_acc_up, Rot_c_i_up, t_c_i_up, P_up


In [37]:
from sympy import symbols, Matrix, cos, sin

# Skew-symmetric matrix function for cross product
def skew_symmetric(vec):
    """Returns the skew-symmetric matrix [vec]_x for a 3x1 vector."""
    return Matrix([
        [0, -vec[2], vec[1]],
        [vec[2], 0, -vec[0]],
        [-vec[1], vec[0], 0]
    ])

# Define symbolic vectors for SE2(3)
chi_r = Matrix(symbols(r'chi_{r1} chi_{r2} chi_{r3}')).reshape(3, 1)
chi_v = Matrix(symbols(r'chi_{v1} chi_{v2} chi_{v3}')).reshape(3, 1)
chi_p = Matrix(symbols(r'chi_{p1} chi_{p2} chi_{p3}')).reshape(3, 1)

# Define the elements in the SE2(3) matrix
R = Matrix(3, 3, symbols('R1:10'))  # 3x3 rotation matrix
v = Matrix(3, 1, symbols('v1:4'))  # 3x1 velocity vector
p = Matrix(3, 1, symbols('p1:4'))  # 3x1 position vector

# Compute the inverse SE2(3) transformation
chi_inverse = Matrix.hstack(R.T, -R.T @ v, -R.T @ p)

# Define the full SE2(3) matrix
chi_epsilon = Matrix.vstack(chi_r, chi_v, chi_p)

# Define angle as the norm of the rotation vector (first 3 elements of state)
angle = (chi_r.norm())  # Use norm of chi_r for rotation
s = sin(angle)
c = cos(angle)

# Calculate a_ and b_
a_ = (1 - c) / angle
b_ = (angle - s / (angle**3))

# Corrected f_error expression
f_error = (R.T @ chi_v +
           a_ * R.T @ (skew_symmetric(chi_r) @ chi_v) +
           b_ * R.T @ (skew_symmetric(chi_r) @ (skew_symmetric(chi_r) @ chi_v)))

# Jacobian of f_error w.r.t. chi_epsilon
Jacobian = f_error.jacobian(chi_epsilon)

# Correct substitution: Set all elements of chi_v and chi_p to 0
subs_dict = {var: 0.00 for var in chi_v}  # Set all chi_v elements to 0
subs_dict.update({var: 0.00 for var in chi_p})  # Set all chi_p elements to 0

# Apply substitution and simplify
Jacobian_simplified = Jacobian.subs(subs_dict)

# Print the final Jacobian
Jacobian_simplified

Matrix([
[0, 0, 0, R1*(-chi_{r2}**2 - chi_{r3}**2)*(sqrt(Abs(chi_{r1})**2 + Abs(chi_{r2})**2 + Abs(chi_{r3})**2) - sin(sqrt(Abs(chi_{r1})**2 + Abs(chi_{r2})**2 + Abs(chi_{r3})**2))/(Abs(chi_{r1})**2 + Abs(chi_{r2})**2 + Abs(chi_{r3})**2)**(3/2)) + R1 + R4*chi_{r1}*chi_{r2}*(sqrt(Abs(chi_{r1})**2 + Abs(chi_{r2})**2 + Abs(chi_{r3})**2) - sin(sqrt(Abs(chi_{r1})**2 + Abs(chi_{r2})**2 + Abs(chi_{r3})**2))/(Abs(chi_{r1})**2 + Abs(chi_{r2})**2 + Abs(chi_{r3})**2)**(3/2)) + R4*chi_{r3}*(1 - cos(sqrt(Abs(chi_{r1})**2 + Abs(chi_{r2})**2 + Abs(chi_{r3})**2)))/sqrt(Abs(chi_{r1})**2 + Abs(chi_{r2})**2 + Abs(chi_{r3})**2) + R7*chi_{r1}*chi_{r3}*(sqrt(Abs(chi_{r1})**2 + Abs(chi_{r2})**2 + Abs(chi_{r3})**2) - sin(sqrt(Abs(chi_{r1})**2 + Abs(chi_{r2})**2 + Abs(chi_{r3})**2))/(Abs(chi_{r1})**2 + Abs(chi_{r2})**2 + Abs(chi_{r3})**2)**(3/2)) - R7*chi_{r2}*(1 - cos(sqrt(Abs(chi_{r1})**2 + Abs(chi_{r2})**2 + Abs(chi_{r3})**2)))/sqrt(Abs(chi_{r1})**2 + Abs(chi_{r2})**2 + Abs(chi_{r3})**2), R1*chi_{r1}*chi_{r